# Project: Analyzing Borrowers’ Risk of Loan Defaulting

__Purpose of project:__


The loan division of a bank had provided data of their customers to find out if a customer's marital status and number of children have an inpact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness. 
The purpose of this report is to analyze the relationship between default rate on loans and  marital status and/or the number of children. This report was used to analyze if there is a increasing default rate on a loan as the number of children increase; as well as if being a widow, widower, divorced, or unmaried has a higher default rate compared to individuals who are married or in a civil partnership. 

Thi report will be considered when building a credit score for a potential customer. A credit score is used to evaluate the ability of a potential borrower to repay their loan. 

__Libraries used in analysis:__


In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

__Data used for credit scoring/ borrower's risk analysis:__

In [2]:
data= pd.read_csv('/datasets/credit_scoring_eng.csv')

## Data exploration

The data in the CSV file was composed of 12 columns ad 21525 rows. In which, each row corresponds to one customer's information on number of children, number of days employed, age, information on education status, information on family status, gender, information on income, and purpose of loan.

**Description of the columns in data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


In [3]:
# first 15 rows of data

data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [4]:
# data DataFrame dimensions 
data.shape

(21525, 12)

### Checking for Null Values in Columns

In [5]:
# info on data DataFrame
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [6]:
# Below is the filtered table with missing values in the the first column with missing data

data[data.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


### Summary of Null Values in Columns


The columns that had missing values include:  days_employed and total_income, which both have 2,174 missing values. 
The rows with missing values in the columns 'days_employed' also contain missing values in the column 'total_income'. This prediction was evaluated by conducting the following lines of code: 

__data[data.days_employed.isnull()]__ , which printed a DataFrame with 2171 rows.                                                 
__data[data.total_income.isnull()]__ , which printed a DataFrame with 2171 rows. To verify the claim about each row with missing values in 'days_employed' column also had missing values in 'total_income' column, the following lines were run: 

__data[data.days_employed.isnull()&data.total_income.isnull()]__, which resulted in a DataFrame with 2174 rows. 
__data[data.days_employed.notnull()&data.total_income.notnull()]__, which resulted in a DataFrame with 19351 rows. 
__data[data.days_employed.isnull()&data.total_income.notnull()]__ , which resulted in an empty DataFrame. 
__data[data.days_employed.notnull()&data.total_income.isnull()]__ ,which resulted in an empty DataFrame. 

These lines of code confirm the claim that each row with missing values in the 'days_employed' column also has missing values in 'total_income' column.


In [7]:
# Applying multiple conditions for filtering data and look at the number of rows in the filtered table.

#size of df where there are not missing values in both 'days_employed' and 'total_income' column
print('length of dataframa where days_employed does not have null values and total_income does not  have null values')
print(len(data[data.days_employed.notnull()&data.total_income.notnull()]))
print()
print('length of dataframa where days_employed has null values and total_income has null values')
print(len(data[data.days_employed.isnull()&data.total_income.isnull()]))
print()
print('length of dataframa where days_employed has null values and total_income does not have null values')
print(len(data[data.days_employed.isnull()&data.total_income.notnull()]))
print()
missing_percentage =round((len(data[data.days_employed.isnull()&data.total_income.isnull()]))*100/len(data),2)
print ('percentage of missing rows:',missing_percentage,'%')

length of dataframa where days_employed does not have null values and total_income does not  have null values
19351

length of dataframa where days_employed has null values and total_income has null values
2174

length of dataframa where days_employed has null values and total_income does not have null values
0

percentage of missing rows: 10.1 %


### Intermediate conclusion



Based on the length of filtered DataFrames, such as: 

__data[data.days_employed.notnull()&data.total_income.notnull()]__ ,

__data[data.days_employed.isnull()&data.total_income.isnull()]__ ,

and __data[data.days_employed.isnull()&data.total_income.notnull()]__ , 

the missing values in columns 'days_employed' and 'total_income' are equal to the number of rows that contain null values in both of these columns. The number of rows that do not contain null values in the column 'days_employed', but have null values in 'total_income' is equal to zero. This means that in each row, in which there is a null value in the 'days_employed' column, there is also a null value in the 'total_income' column. 



Since 2171 missing values out of 21525 is approximately 10% of the total values, then the missining values represent a significant amount of the total values. One characteristic about the missing values is that If the customer has a value of graduate school, then there was not a missing value. 

Below is the original distribution of missing value rate by categories of education, income type, and family status. 

### Distrubution of Missing values per categories of education, income type, and family status. 

Based on the distribution of missing values per education type, people who received up to a secondary education had the greatest missing rate, with 70.8% of the missing values for days employed and total income. Consequently, people who received up to a bachelor's degree education had 25% of the missing values days employed and total income. Meanwhile, people who received a graduate degree education, they did not present any missing values. 

Similarly, the distribution for missing values per family status had the majority of the missing values in people who were married, with 56.9% of the missing values in total income and days employed. People in a civil partnership and who were unmarried had 20.3% and 13.3% of the missing data, respectively, on days employed and missing values. 

For the missing values versus income type distribution, there was a similar distribution compated to the missing values versus income type distribution; people who were employed, had a business and were retired represented 50.8%, 23.4%, and 19%, respectively, of the missing values for days employed and total income.

In [8]:
# Checking distribution
nan_data = data[data['days_employed'].isnull()]
nan_data['education']=data.education.str.lower()

print('Distribution of null values by categories of income type')
print (nan_data['income_type'].value_counts(normalize = True))
print()
print('Distribution of null values by categories of education')
print (nan_data['education'].value_counts(normalize = True))
print()
print('Distribution of null values by categories of family status')
print (nan_data['family_status'].value_counts(normalize = True))


Distribution of null values by categories of income type
employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

Distribution of null values by categories of education
secondary education    0.708372
bachelor's degree      0.250230
some college           0.031739
primary education      0.009660
Name: education, dtype: float64

Distribution of null values by categories of family status
married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64


/tmp/ipykernel_76/3023196680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_data['education']=data.education.str.lower()


### Possible reasons for missing values in data


Based on the distribution for missing data per education, family status, and income type, the people who have no longer a partner (divorced and widow/widower) have the smallest percentages of missing values. Similarly, people who have never had a partner (unmarried) only account for 13.3% of the missing values for days employed and total income. On the other hand, people who are involved with a partner, either a civil partnership or married, have the largest percentages of missing data. Based on this pattern, one possible reason for missing values in data could be the marital status of the person. In a similar note, people who are not currently employed or working for a company (students, people on paternity/maternity leave, entrepreneurs, and civil servants) present small percentages of missing data; however, people who are currently or were previously employed (retirees, people owning a business, or employees) have the largest percentages of missing data. In contrast with income type and family status, the distribution for missing data versus education level does not present a pattern that would suggest a reason for the missing values in data. For instance, people who completed a graduate level education have 0% of missing data; however, people completed up to a bachelors degree have the second biggest percentage of missing data. If there was a pattern in education, we would predict that as education level increases, the percentage of missing data would decrease. With that said, the distribution of missing values per education level does not present a pattern with percentage of missing data. 

### Intermediate conclusion

To test how the distribution of missing values for type of education, income type, and family status differs from the original distribution in the original dataset, the coding cell below summarizes the percentages of education, income and family status type from the data. As we observed, the percentages of education type did not present a significant difference between distribution of data with missing values and original data. For instance, people who were employees represented the highest percentage in the original dataset, followed by people in business and retirees; These three income types represented 51.6%, 23.7% and 17.8%, respectively, for the original data. A similar distribution for employees, business people, and retirees was shown in the dataset with missing values, with percentage values of 50.8%, 23.4%, and 19%. 

Similarly to income type, the distribution of education type did not show a significant difference between the percentage values of the original dataset and the data with missing values. In the original dataset, the education types that had the highest percentage values in the distribution wer secondary education and bachelor's degree, with values of 70.6% and 24.4%, respectively. In the dataset with missing values, secondary education and bachelor's degree had the highest values in the distrubtion, with values of 70.8% and 25%, respectively. 

In comparison to education type and income type, the distribution of family status also did not have a significant difference between the percentage values of the original dataset and the dataset with missing values. In both distributions, the people who were married or in a civil partnership had the highest percentage values. In the original dataset, the percentage values for people who were married and in a civil partnership represented 57.5% and 19.4%, respectively. Similarly, in the dataset with missing values, people who were married and in a civil partnership represented 56.8% and 20.3%, respectively. 

In [9]:
# Checking the distribution in the whole dataset

original_dataset = data
original_dataset['education']=data.education.str.lower()
print('Distribution of values per categories in income type')
print (original_dataset['income_type'].value_counts(normalize = True))
print()
print('Distribution of values per categories in education')
print (original_dataset['education'].value_counts(normalize = True))
print()
print('Distribution of values per categories in family status')
print (original_dataset['family_status'].value_counts(normalize = True))

Distribution of values per categories in income type
employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

Distribution of values per categories in education
secondary education    0.707689
bachelor's degree      0.244367
some college           0.034564
primary education      0.013101
graduate degree        0.000279
Name: education, dtype: float64

Distribution of values per categories in family status
married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64


In [10]:
# Checking for other patterns - explain which

print('Distribution of values per categories in debt column')
print (original_dataset['debt'].value_counts(normalize = True))
print()
print('Distribution of missing values per categories in debt column')
print (nan_data['debt'].value_counts(normalize = True))
print()
print('Distribution of values per categories in gender column')
print (original_dataset['gender'].value_counts(normalize = True))
print()
print('Distribution of missing values per categories in gender column')
print (nan_data['gender'].value_counts(normalize = True))

Distribution of values per categories in debt column
0    0.919117
1    0.080883
Name: debt, dtype: float64

Distribution of missing values per categories in debt column
0    0.921803
1    0.078197
Name: debt, dtype: float64

Distribution of values per categories in gender column
F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64

Distribution of missing values per categories in gender column
F    0.682613
M    0.317387
Name: gender, dtype: float64


### Conclusion

Overall, the distribution of missing values shows a particular pattern in which people who are married or in a civil partnership have the highest number of missing values in both the original dataset and the dataset with missing values. The percentage values for family status did not differ significantly in both distributions. On the same note, people who completed up to a secondary eduction or a bachelor's degree represented the highest percentage values in both the original dataset and the dataset with missing values. The percentage values for education did not differ significantly in both distributions. In comparison to education type and family status, income type also did not present a significant difference between the distributions of the original dataset and the dataset with missing values. In the previous coding cell, the distribution of debt status and gender type also did not present a significant difference between the distributions of the original dataset and the dataset with missing values. Both the original dataset and the dataset with missing values showed that about 92% of the people in both datasets paid their loan; and that females acount for 66% and 68% of percent values in the original dataset and the dataset with missing values, respectively. So overall conclusion here, the missing values is probably a random error. There is no pattern or corelation here.

## Data transformation

In this section, we transfomed the data to ensure that there are not errors in the values of all the columns. 

### education column 

The column 'education' had repeated categories in different formats: uppercase and lowercase. To solve this problem, all the categories were converted to lowercase.

In [11]:
# all values in education column to check if and what spellings will need to be fixed
print(data.education.unique())

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [12]:
# Fix of the registers 
data['education']=data.education.str.lower()

In [13]:
# Checking all the values in the column to make sure were fixed
print(data.education.unique())

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


### children column

The 'children' column had values 0f (-1) and 20, which are nonsense values. 

In [14]:
# the distribution of values in the `children` column
print(data.children.unique())

[ 1  0  3  2 -1  4 20  5]


The negative value (-1) is most likely an input error.
Similarly, the value 20 could also be a data entry error. Since rows with these values only account for less than 1% (0.57%) of the data, the rows with these values were dropped. We are uncertain about the occurrence of these errors in number of children. 

In [15]:
#dropping rows with children equal to (-1) and 20 values
print(round(len(data.loc[(data.children==20) | (data.children==-1)])*100/len(data),2))
data= data.loc[(data['children'] != 20) & (data['children'] != -1)]

0.57


In [16]:
# Checking the `children` column again to make sure it's all fixed
print(data.children.unique())

[1 0 3 2 4 5]


### days_employed column

The values in the `days_employed` column contained negative values. These values could be a result of using a formula in the incorrect order to calculate amount of days. Since the general formula to calculate the amount of days employed is (final date of employment-initial start date), the result of using this formula in the incorrect order would result in a negative input, with the same magnitude as if were used in the correct order. Therefore, the values in the `days_employed` were fixed by replacing them with their magnitude. 

In [17]:
# problematic data in `days_employed`
print(data.days_employed.unique())

[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]



The number of days are floats and some values are negative; the values were fixed by replacing them with the value of their magnitude. 
The unknown values will be evaluated and rounded as integers in the `Restoring values in days_employed` section of the report. 

In [18]:
# Addressing the problematic values
data['days_employed']=data.days_employed.abs()

/tmp/ipykernel_76/866075963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['days_employed']=data.days_employed.abs()


In [19]:
# Checking the result to make sure it's fixed
print(data.days_employed.unique())

[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


### dob_years column


The `dob_years` column had values of zero that are nonsense in data about loans, specially for rows in which the individual was married, divorced, or married. 

In [20]:
# Checking the `dob_years` for suspicious values and count the percentage

sorted(data.dob_years.unique())
data.loc[data['dob_years']==0]
# data.income_type.unique()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate




Since the number of rows that have values of zero for age only account for 0.47% of the data, then these rows will be dropped because we don't know the source of error. Dropping these rows will not be a significant influence in our overall results. 


In [21]:
#dropping the rows with age value of 0
len(data.loc[data['dob_years']==0])*100/len(data) 
data.drop(data.loc[data.dob_years==0].index, inplace= True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [22]:
# Checking the result to make sure it's fixed
data.dob_years.unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

### family_status column

The values in the `family_status` column have spaces. Each of the unique values that had spaces in them were formatted in snake_case format. 

In [23]:
# values for the column
data.family_status.unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [24]:
# Addressing the problematic values in `family_status`

data.family_status.replace('civil partnership','civil_partnership', inplace=True)
data.family_status.replace('widow / widower','widow_or_widower', inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [25]:
# Checking the result to make sure it's fixed
data.family_status.unique()


array(['married', 'civil_partnership', 'widow_or_widower', 'divorced',
       'unmarried'], dtype=object)

### gender column

The `gender` column had 'XNA' values, which could mean that the gender was not inputted; there was not information about the gender of the person; or the gender which the person identified as was not available as an input. There is only one row in the data that contains this value for gender. Dropping this row will not be a significant influence in our analysis. 

In [26]:
# values in the column
print(data.gender.unique())
len(data.loc[data.gender=='XNA'])

['F' 'M' 'XNA']


1

In [27]:
# Dropping row with value 'XNA' in gender column
data.drop(data.loc[data.gender=='XNA'].index, inplace= True)

In [28]:
# Checking the result to make sure it's fixed, if problem exists
data.gender.unique()

array(['F', 'M'], dtype=object)

### income_type column

The values in the `income_type` column have spaces. Each of the unique values that had spaces in them were formatted in snake_case format.

In [29]:
# values in the column
data.income_type.unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [30]:
# Addressing the problematic values
data.income_type.replace('civil servant','civil_servant',inplace=True)
data.income_type.replace('paternity / maternity leave','paternity_or_maternity_leave',inplace=True)

In [31]:
# Checking the result to make sure it's fixed

data.income_type.unique()

array(['employee', 'retiree', 'business', 'civil_servant', 'unemployed',
       'entrepreneur', 'student', 'paternity_or_maternity_leave'],
      dtype=object)

### checking for duplicates

__Duplicated data__
The duplicated rows were dropped because it could mean that the information was inputted more than once. There were in total of 54 duplicates, and their contribution to the data is less than 1%; therefore, dropping these rows will not significantly influence our analysis. 

In [32]:
#Checking duplicates
duplicate_data=data[data.duplicated(keep='last')]
print(len(duplicate_data)*100/len(data))

0.33331768461574574


In [33]:
# Addressinf the duplicates
data.drop_duplicates(keep=False, inplace=True)

/tmp/ipykernel_76/1515868590.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(keep=False, inplace=True)


In [34]:
# Last check whether there are any duplicates
duplicate_data=data[data.duplicated(keep='last')]
print(duplicate_data)

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [35]:
# size of the dataset after first manipulations 
print(data.info())
print(data[data.days_employed.notnull()&data.total_income.isnull()])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21164 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21164 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21164 non-null  int64  
 3   education         21164 non-null  object 
 4   education_id      21164 non-null  int64  
 5   family_status     21164 non-null  object 
 6   family_status_id  21164 non-null  int64  
 7   gender            21164 non-null  object 
 8   income_type       21164 non-null  object 
 9   debt              21164 non-null  int64  
 10  total_income      19149 non-null  float64
 11  purpose           21164 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB
None
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpo


After dropping duplicates, the number of rows from the original data decreased by 1.67% . Again, by using the line of code: data[data.days_employed.notnull()&data.total_income.isnull()] , we observed that for each missing value in 'days_employed', there is a missing value in the 'total_income' column. 

# Working with missing values

### Restoring missing values in `total_income`

The age_categorization function was created to create the `age_category` column to describe the age range that each person belongs to. These age ranges were used as indices in the `median_income_pivot`, which examines the median income per income_type and education.

In [36]:
#function that calculates the age category
def age_categorization(row):
    age=row['dob_years']
 
    if age < 30:
        return('20s')
    if 30<= age <40:
        return('30s')
    if 40<= age <50:
        return('40s')
    if 50<= age <60:
        return('50s')
    if 60<= age <70:
        return('60s')
    if age >=60:
        return('70s_or_older')

In [37]:
# Testing if the function works
row_values=[45]
row_columns=['dob_years']
row=pd.Series(data=row_values,index=row_columns)
age_categorization(row)

'40s'

In [38]:
# Creating new column based on function
data['age_category']=data.apply(age_categorization,axis=1)

/tmp/ipykernel_76/789463299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['age_category']=data.apply(age_categorization,axis=1)


In [39]:
# Checking how values in the new column
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40s
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30s
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30s
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30s
4,0,340266.072047,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50s


The `no_nan_values` table was used to find the distribution of the 'total_income' column without having missing data.

In [40]:
#table without missing values in the `Total_income` column. 
no_nan_values= data.dropna()
no_nan_values

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40s
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30s
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30s
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30s
4,0,340266.072047,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil_partnership,1,F,business,0,35966.698,housing transactions,40s
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60s
21522,1,2113.346888,38,secondary education,1,civil_partnership,1,M,employee,1,14347.610,property,30s
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30s


#### Distribution of values of factors that affect income
Below are the distributions of factors that affect income in the following pivot tables. 

In [41]:
# the distribution values for income based on age_category
no_nan_values.groupby('age_category')['total_income'].agg(['min','median','mean','max'])

,min,median,mean,max
age_category,,,,
20s,4494.861,22765.2935,25544.231203,131588.163
30s,3392.845,24699.5815,28314.525654,352136.354
40s,4036.463,24772.3820,28575.427654,362496.645
50s,3306.762,22195.8470,25807.707523,195686.797
60s,3471.216,19811.3260,23236.985508,274402.943
70s_or_older,3595.641,18751.3240,20125.658331,57508.032


In [42]:
# the distribution values for income based on education
no_nan_values.groupby('education')['total_income'].agg(['min','median','mean','max'])

,min,median,mean,max
education,,,,
bachelor's degree,5148.514,28086.5425,33197.258790,362496.645
graduate degree,15800.399,25161.5835,27960.024667,42945.794
primary education,4049.374,18741.9760,21144.882211,78410.774
secondary education,3306.762,21832.2410,24594.390815,276204.162
some college,5514.581,25618.4640,29028.844227,153349.533


In [43]:
# the distribution values for income based on income type
no_nan_values.groupby('income_type')['total_income'].agg(['min','median','mean','max'])

,min,median,mean,max
income_type,,,,
business,4592.450,27594.6410,32424.420789,362496.645
civil_servant,4672.012,24076.1150,27336.442546,145672.235
employee,3418.824,22815.1035,25822.872585,276204.162
entrepreneur,79866.103,79866.1030,79866.103000,79866.103
paternity_or_maternity_leave,8612.661,8612.6610,8612.661000,8612.661
retiree,3306.762,18959.6260,21950.722935,117616.523
student,15712.260,15712.2600,15712.260000,15712.260
unemployed,9593.119,21014.3605,21014.360500,32435.602


In [44]:
# the distribution values for income based on genderr
no_nan_values.groupby('gender')['total_income'].agg(['min','median','mean','max'])

,min,median,mean,max
gender,,,,
F,3306.762,21465.6375,24666.721477,274402.943
M,3392.845,26828.2450,30913.888696,362496.645


#### intermediate conclusion
Based on the distribution of income type, median will be used because mean is affected by outliers. Although gender is an influential factor in total_income, the type of job or source of income, education and age have considerably more difference in total income. The median values values of total_income for the pivot tables below were used, since our distributions showed outliers. 

The `median_income_pivot` categorized the median income per age, income type and education level. However, there are some null values in this table. To take into account these null values, the `income_type_education_pivot` table was used as a second resource to use in a situation that there's a null value in `median_income_table`, for a person who has a missing total income value. The `income_type_education_pivot` table categorizes the median income per income type and education. Similarly to `median_income_pivot`, the `income_type_education_pivot`also presents some null values for their indices. As a third resource, we used the `income_type_pivot` table, which categorizes the median total income per income_type. 

The `filling_missing_income_in_data` function was created to fill in the missing total income values in the 'total_income' column depending on the person's age category, income type and education. Inside the function, each of these pivot tables were assigned a variable name that describes the ranking to use them as a resource to find missing data. For instance, the `median_income_pivot` table was assigned the `first_resource` variable name. The `filling_missing_income_in_data` function also prevents error for people whose income type is not listed in the `first_resource` pivot table. For instance, people who had mising income, and were in their 50s and were entrepreneurs did not have a category in for entrepreneurship in the `filling_missing_income_in_data`. Therefore, try and except attributes were used to look in the `second_resource` table and prevent errors. 

In [45]:
median_income_pivot=no_nan_values.pivot_table(
index=['age_category','income_type'],
columns='education',
values='total_income',
aggfunc='median', fill_value='missing')


income_type_education_pivot=no_nan_values.pivot_table(
index='income_type',
columns='education',
values='total_income',
aggfunc='median', fill_value='missing')


income_type_pivot=no_nan_values.pivot_table(
index='income_type',
values='total_income',
aggfunc='median', fill_value='missing')

median_income_pivot

education                                 bachelor's degree graduate degree  \
age_category income_type                                                      
20s          business                             28250.284         missing   
             civil_servant                       24483.1205         missing   
             employee                             24142.603         missing   
             entrepreneur                         79866.103         missing   
             retiree                              34394.128         missing   
             student                               15712.26         missing   
30s          business                            32639.6155         missing   
             civil_servant                       27956.2445       17822.757   
             employee                             26773.885       18551.846   
             paternity_or_maternity_leave           missing         missing   
             retiree                              30117.883         missing   
             unemployed                             missing         missing   
40s          business                             34592.045         missing   
             civil_servant                        28425.481         missing   
             employee                             27497.588       31771.321   
             retiree                             31828.7485         missing   
             unemployed                           32435.602         missing   
50s          business                             35415.771         missing   
             civil_servant                        28366.568         missing   
             employee                             27471.936       42945.794   
             retiree                             22805.7985         missing   
60s          business                            30849.5425         missing   
             civil_servant                        27117.841         missing   
             employee                            27969.0405         missing   
             retiree                             23522.6545       28334.215   
70s_or_older business                             36808.968         missing   
             civil_servant                        57508.032         missing   
             employee                               missing         missing   
             retiree                              22009.315         missing   

education                                 primary education  \
age_category income_type                                      
20s          business                             24210.196   
             civil_servant                        30563.383   
             employee                            24583.5965   
             entrepreneur                           missing   
             retiree                                missing   
             student                                missing   
30s          business                             20030.226   
             civil_servant                        21150.696   
             employee                             19546.341   
             paternity_or_maternity_leave           missing   
             retiree                              14138.866   
             unemployed                             missing   
40s          business                             27929.098   
             civil_servant                        78410.774   
             employee                            21132.3895   
             retiree                              15708.433   
             unemployed                             missing   
50s          business                             23066.131   
             civil_servant                       12709.2755   
             employee                             14599.258   
             retiree                              17104.925   
60s          business                               missing   
             civil_servant            

In [46]:
income_type_education_pivot

education,bachelor's degree,graduate degree,primary education,secondary education,some college
income_type,,,,,
business,32285.664,missing,21887.825,25441.23,28778.744
civil_servant,27564.459,17822.757,23734.287,21864.475,25694.775
employee,26587.423,31771.321,20159.186,21841.813,24209.43
entrepreneur,79866.103,missing,missing,missing,missing
paternity_or_maternity_leave,missing,missing,missing,8612.661,missing
retiree,23030.247,28334.215,16415.785,18372.071,19221.903
student,15712.26,missing,missing,missing,missing
unemployed,32435.602,missing,missing,9593.119,missing


In [47]:
income_type_pivot

,total_income
income_type,
business,27594.6410
civil_servant,24076.1150
employee,22815.1035
entrepreneur,79866.1030
paternity_or_maternity_leave,8612.6610
retiree,18959.6260
student,15712.2600
unemployed,21014.3605


In [48]:
#  function used for filling in missing values
def filling_missing_income_in_data(row):
    income_type=row['income_type']
    total_income=row['total_income']
    education=row['education']
    age_category=row['age_category']
    
    first_resource=median_income_pivot
    second_resource=income_type_education_pivot
    third_resource= income_type_pivot
    
    if total_income != 'missing':
        return total_income
    
    try:
        if total_income =='missing':
            if first_resource.loc[(age_category,income_type),education]=='missing':
                if second_resource.loc[income_type,education]=='missing':
                    return third_resource.loc[income_type]
                return second_resource.loc[income_type,education]
            return first_resource.loc[(age_category,income_type),education]
    
    except:
        if total_income =='missing':
            if second_resource.loc[income_type,education]=='missing':
                return third_resource.loc[income_type]
            return second_resource.loc[income_type,education]

In [49]:
# Checking if the function works

row_values=['20s','business',"bachelor's degree", 'missing']
row_columns=['age_category','income_type','education','total_income']
row=pd.Series(data=row_values,index=row_columns)
filling_missing_income_in_data(row)


28250.284

In [50]:
# Applying it to every row
data.total_income.fillna('missing',inplace=True)
data['total_income']=data.apply(filling_missing_income_in_data,axis=1)
data['total_income']=data['total_income'].round(2)


/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/tmp/ipykernel_76/2475972195.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income']=data.apply(filling_missing_income_in_data,axis=1)
/tmp/ipykernel_76/2475972195.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [51]:
# Checking if we got any errors
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.10,purchase of the house,40s
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.80,car purchase,30s
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.75,purchase of the house,30s
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.57,supplementary education,30s
4,0,340266.072047,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.57,to have a wedding,50s
5,0,926.185831,27,bachelor's degree,0,civil_partnership,1,M,business,0,40922.17,purchase of the house,20s
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.16,housing transactions,40s
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.83,education,50s
8,2,6929.865299,35,bachelor's degree,0,civil_partnership,1,F,employee,0,15337.09,having a wedding,30s
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.15,purchase of the house for my family,40s


In [52]:
# Checking the number of entries in the columns

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21164 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21164 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21164 non-null  int64  
 3   education         21164 non-null  object 
 4   education_id      21164 non-null  int64  
 5   family_status     21164 non-null  object 
 6   family_status_id  21164 non-null  int64  
 7   gender            21164 non-null  object 
 8   income_type       21164 non-null  object 
 9   debt              21164 non-null  int64  
 10  total_income      21164 non-null  float64
 11  purpose           21164 non-null  object 
 12  age_category      21164 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


###  Restoring values in `days_employed`

The parameter used to to categorize the distribution of number of days employed was the income_type column. The values used to replace missing values were median values because the there were outliers in the distribution. 

In [53]:
# Distribution of `days_employed` medians based on income type
days_employed_age_group=data.groupby('income_type')['days_employed'].agg(['min','median','mean','max'])
days_employed_age_group

,min,median,mean,max
income_type,,,,
business,30.195337,1555.993659,2119.512866,17615.563266
civil_servant,39.954170,2672.903939,3392.119263,15193.032201
employee,24.141633,1573.791064,2325.740892,18388.949901
entrepreneur,520.848083,520.848083,520.848083,520.848083
paternity_or_maternity_leave,3296.759962,3296.759962,3296.759962,3296.759962
retiree,328728.720605,365269.100414,365037.486121,401755.400475
student,578.751554,578.751554,578.751554,578.751554
unemployed,337524.466835,366413.652744,366413.652744,395302.838654


In [54]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
def age_employed_function(row):
    income_type=row['income_type']
    days_employed=row['days_employed']
    
    if days_employed =='missing':
        return days_employed_age_group['median'][income_type]
    return days_employed

In [55]:
# Checking that the function works
row_values=['business','missing']
row_columns=['income_type','days_employed']
row=pd.Series(data=row_values,index=row_columns)
age_employed_function(row)

1555.993659260394

In [56]:
# Applying function to the income_type

data.days_employed.fillna('missing',inplace=True)
data['days_employed']=data.apply(age_employed_function,axis=1)

data.loc[data['days_employed']=='missing']

data['days_employed']=data['days_employed'].astype('int')


/opt/conda/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/tmp/ipykernel_76/2630126215.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['days_employed']=data.apply(age_employed_function,axis=1)
/tmp/ipykernel_76/2630126215.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [57]:
# Check if function worked
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.10,purchase of the house,40s
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.80,car purchase,30s
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.75,purchase of the house,30s
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.57,supplementary education,30s
4,0,340266,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.57,to have a wedding,50s


In [58]:
# Checking  the entries in all columns to make sure we fixed all missing values

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21164 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21164 non-null  int64  
 1   days_employed     21164 non-null  int64  
 2   dob_years         21164 non-null  int64  
 3   education         21164 non-null  object 
 4   education_id      21164 non-null  int64  
 5   family_status     21164 non-null  object 
 6   family_status_id  21164 non-null  int64  
 7   gender            21164 non-null  object 
 8   income_type       21164 non-null  object 
 9   debt              21164 non-null  int64  
 10  total_income      21164 non-null  float64
 11  purpose           21164 non-null  object 
 12  age_category      21164 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.8+ MB


## Categorization of data

To test the hypothesis, the values in the 'purpose' column were categorized in the following categories: 'housing', 'vehicle', 'education', 'event_or_ceremony', or 'business'. The categories were assigned based on the description in the 'pupose' column. 

In [59]:
# Printing the values for your selected data for categorization
data['purpose'].unique().tolist()

['purchase of the house',
 'car purchase',
 'supplementary education',
 'to have a wedding',
 'housing transactions',
 'education',
 'having a wedding',
 'purchase of the house for my family',
 'buy real estate',
 'buy commercial real estate',
 'buy residential real estate',
 'construction of own property',
 'property',
 'building a property',
 'buying a second-hand car',
 'buying my own car',
 'transactions with commercial real estate',
 'building a real estate',
 'housing',
 'transactions with my real estate',
 'cars',
 'to become educated',
 'second-hand car purchase',
 'getting an education',
 'car',
 'wedding ceremony',
 'to get a supplementary education',
 'purchase of my own house',
 'real estate transactions',
 'getting higher education',
 'to own a car',
 'purchase of a car',
 'profile education',
 'university education',
 'buying property for renting out',
 'to buy a car',
 'housing renovation',
 'going to university']

The 'purpose_categ' function was used to insert a purpose category value in the 'pupose_category' column. The function was created based on unique values in the 'purpose' column. 

In [60]:
def purpose_categ(purpose):
    
    if 'car' in purpose:
        return 'car'
    if 'wedding' in purpose:
        return 'event_or_ceremony'
    if 'educat' in purpose or 'university' in purpose:
        return 'education'
    return 'housing_or_property'

In [61]:
# unique values in 'purpose_category' column

data['purpose_category'] = data['purpose'].apply(purpose_categ)
data.purpose_category.unique()

/tmp/ipykernel_76/1045536722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['purpose_category'] = data['purpose'].apply(purpose_categ)


array(['housing_or_property', 'car', 'education', 'event_or_ceremony'],
      dtype=object)

In [62]:
# Looking through all the numerical data in your selected column for categorization
income_purpose_pivot= data.pivot_table(index='income_type',columns='purpose_category',values='total_income',aggfunc='mean')
income_purpose_pivot

purpose_category,car,education,event_or_ceremony,housing_or_property
income_type,,,,
business,32271.168636,30363.907722,32253.594427,32503.224033
civil_servant,26714.221802,27241.318775,27636.418333,26979.051119
employee,25460.926038,25387.885897,24957.380527,25763.479377
entrepreneur,NaN,NaN,79866.100000,79866.100000
paternity_or_maternity_leave,8612.660000,NaN,NaN,NaN
retiree,21457.933804,21851.113225,22277.578978,21606.522082
student,NaN,NaN,NaN,15712.260000
unemployed,NaN,NaN,NaN,21014.360000


In [63]:
# Getting summary statistics for the column

purpose_pivot= data.pivot_table(index='age_category',columns='purpose_category',values='total_income',aggfunc='mean')

data.groupby('purpose_category')['total_income'].agg(['mean','median', 'max'])


,mean,median,max
purpose_category,,,
car,26468.444297,22781.770,228469.51
education,26060.893936,22705.800,276204.16
event_or_ceremony,26330.949596,22881.195,273809.48
housing_or_property,26716.673076,23195.505,362496.64


## Checking the Hypotheses


**Question: Is there a correlation between having children and paying back on time?**

In [64]:
# Check the children data and paying back on time

children=[0,1,2,3,4,5]
default_children_rate=[]

for x in children:
    default_total=data.loc[(data['debt']==1) & (data['children']==x)]
    default_length_total=len(default_total)
    
    all_total=data.loc[data['children']==x]
    length_all_total=len(all_total)
    
    default_r=round((default_length_total/length_all_total)*100,2)
    default_children_rate.append(default_r)



default_rate_children={'children':children,
                 'default_rate_%': default_children_rate
}

default_rate_children_table=pd.DataFrame(default_rate_children)
default_rate_children_table

,children,default_rate_%
0,0,7.57
1,1,9.22
2,2,9.53
3,3,8.23
4,4,9.76
5,5,0.00


**Conclusion**

Based on the default_rate_children_table table, the default rate is higher for people who have four children, followed by people who have two and one children, with a default rate of 9.8%, 9.5%, and 9.2%, repectively. On the other hand, people who had five children presented the lowest default rate with a 0% value. People who had no children or three presented a 7.6% and 8.2% default rate, respectively. Overall, there was not a signinficant difference, on average, greater than 9.8% default rate among the number of children. 


**Question: Is there a correlation between family status and paying back on time?**

In [65]:
# Check the family status data and paying back on time


family_status= ['married', 'civil_partnership', 'widow_or_widower', 'divorced', 'unmarried']

family_status_default_count=[]
for x in family_status:
    default_total=data.loc[(data['debt']==1) & (data['family_status']==x)]
    default_length_total=len(default_total)
    
    all_total=data.loc[data['family_status']==x]
    length_all_total=len(all_total)
    
    default_r=round((default_length_total/length_all_total)*100,2)
    family_status_default_count.append(default_r)

print(family_status_default_count)
# Calculating default-rate based on family status

default_rate_family_dictionary={'family_status':family_status,
                 'default_rate_%': family_status_default_count
}

default_rate_family=pd.DataFrame(default_rate_family_dictionary)
default_rate_family.sort_values('default_rate_%',ascending=False)

[7.58, 9.37, 6.56, 7.12, 9.79]


,family_status,default_rate_%
4,unmarried,9.79
1,civil_partnership,9.37
0,married,7.58
3,divorced,7.12
2,widow_or_widower,6.56


**Conclusion**

Based on the default_rate_family table, people who were unmarried or involved in a civil partnership presented the highest default rates, with default rate values of 9.8% and 9.4% respectively. With that said, the people who were widowers/widows,  divorced, or married had the lowest three ranks in default rate, with values of 6.6%, 7.1%, and 7.6% respectively. 

**Is there a correlation between income level and paying back on time?**

In [66]:
### Check the income level data and paying back on time

total_income=data[['total_income','debt']]

def income_level(row):
    total_income=row['total_income']
    
    if total_income >0 and total_income<51000:
        return '0K-50K'
    if total_income >=51000 and total_income<101000:
        return '51K-100K'
    if total_income >=101000 and total_income<151000:
        return '101K-150K'
    if total_income >=151000 and total_income<201000:
        return '151K-200K'
    if total_income >=201000 and total_income<251000:
        return '201K-250K'
    if total_income >=251000 and total_income<301000:
        return '251K-300K'
    if total_income >=301000: 
        return '301K_and_up'
    
    
total_income['income_level_category']=total_income.apply(income_level,axis=1)


total_income_category=['0K-50K','51K-100K','101K-150K','151K-200K','201K-250K', '251K-300K','301K_and_up']

total_income_default_rate=[]
for x in total_income_category:
    default_total=data.loc[(total_income['debt']==1) & (total_income['income_level_category']==x)]
    default_length_total=len(default_total)
    
    all_total=data.loc[total_income['income_level_category']==x]
    length_all_total=len(all_total)
    
    default_r=round((default_length_total/length_all_total)*100,2)
    total_income_default_rate.append(default_r)

loan_default_income_dictionary={'income_level':total_income_category,'default_rate_%':total_income_default_rate
    
}

loan_default_income_level_df=pd.DataFrame(loan_default_income_dictionary)
loan_default_income_level_df.sort_values('default_rate_%',ascending=False)

/tmp/ipykernel_76/623441471.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_income['income_level_category']=total_income.apply(income_level,axis=1)


,income_level,default_rate_%
6,301K_and_up,50.00
0,0K-50K,8.23
1,51K-100K,6.93
3,151K-200K,5.88
2,101K-150K,5.71
4,201K-250K,0.00
5,251K-300K,0.00


**Conclusion**

Based on the loan_default_income_level_df table, the income levels between 301,000 and above presented the highest default rate with a 50% value. On the other hand, the income levels that presented te lowest default rate were people who earned between 201,000 and 300,000 monthly. With that said, the rest of the income levels, with exception of 201,000 and above, presented range of default rate between 5.7% and 8.2%. 

**How does credit purpose affect the default rate?**

In [67]:
# Check the percentages for default rate for each credit purpose and analyze them

purpose_category=['housing_or_property', 'car', 'education', 'event_or_ceremony']

default_rate=[]
for x in purpose_category:
    default_total=data.loc[(data['debt']==1) & (data['purpose_category']==x)]
    default_length_total=len(default_total)
    
    all_total=data.loc[data['purpose_category']==x]
    length_all_total=len(all_total)
    
    default_r=round((default_length_total/length_all_total)*100,2)
    default_rate.append(default_r)

purpose_default_rate_dictionary={'purpose_category':purpose_category,'default_rate_%':default_rate}
purpose_default_rate_df=pd.DataFrame(purpose_default_rate_dictionary)
purpose_default_rate_df

,purpose_category,default_rate_%
0,housing_or_property,7.28
1,car,9.34
2,education,9.32
3,event_or_ceremony,7.95


**Conclusion**

Based on the `purpose_default_rate_df` table, the purpose category that presented the highest default rate was vehicles with a 9.34%, followed by education with a default rate of 9.32%. The rest of the categories: housing_or_property and event_or_ceremony, presented range in default rate between 7.28% and 7.95%, respectively. 

# General Conclusion 


As observed in the `Checking the hypothesis` section, the distribution of default rate per number of children did not show a general trend of default rate increase as number of children increases. This does not support the hypothesis we had at the beginning of the analysis. The distribution showed that people who have 1,2, or 4 children have the highest percentage probability to a loan with 9.2%, 9.5%, and 9.8% default rate, respectively. In contrast, people who have 5 children have the lowest default rate to pay loan. The default rate descreases gradually as the number of children increase, with the exception of people who have 3 children, as they have a 8.2% default rate, but the default rate decreases to 0% when we get to 5 children. Overall, the default rate per number of children did not vary significantly for people who have 0 to 4 children. 

In addition, the distribution of defaul rate per marital status does not support our initial hypothesis of the analysis. We did not see a higher default rate to a loan for widowers/widows, unmarried or divorced. In fact, people who were unmarried presented the highest default rate, above people who were married or in a civil partnership. One aspect we observed in this analysis is that divorced and widowers/widows have the lowest two rankings in defult rate, among the family status categories. However, there is not siginificant different, on average, greater than 3.3% default rate among the family status categories. 